# Setings

Bibliotecas usadas:

- datetime
- pandas
- matplotlib

- sys
- json
- glob

- webvtt

Outros:

- Youtube API (https://developers.google.com/youtube/v3)
- Youtube-dl (https://youtube-dl.org/)

## Configurações básicas 

- Bibliotecas usadas
- Diretórios e locais de trabalho


In [4]:
# Bibliotecas usadas

"""
Caso não tenha algumas das bibliotecas instaladas digitar "!pip install". Exemplo: !pip install os. 
Exceção para os casos YoutubeAPI e Youtube-dl. Nesses casos, acessar os links indicados e verificar os documentos disponíveis. 
"""

import os
import pandas as pd
from matplotlib import pyplot as plt

import sys
import datetime
import json
import glob

import webvtt

In [5]:
# Diretórios e locais de trabalho
## dirbase = os.chdir("") incluir sua base de trabalho

cwd = os.getcwd()
print(cwd)

/Users/dumoura/Dev/PDev/PoliticalRemix/Lives_da_Semana/Notebook_IngDef/Bolsonorao_Channel_Parsing


In [6]:
# Definir diretórios e locais de trabalho

BASE_DIR = os.path.dirname(cwd) # base de trabalho
DATA_DIR = os.path.join(BASE_DIR, "data") # dados gerais levantados durante projeto
META_DIR = os.path.join(BASE_DIR, "metadados") #metadados levantados durante projeto
CACHE_DIR = os.path.join(BASE_DIR, "cache") # material em processo - pode ser apagado ao final, caso julgue necessário
SUB_DIR = os.path.join(CACHE_DIR, "subs") # local de trabalho para processamento de legenda
VID_DIR = os.path.join(CACHE_DIR, "vids") # local de trabalho para processamento de legenda

# Criar diretórios e locais de trabalho

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(META_DIR, exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(SUB_DIR, exist_ok=True)
os.makedirs(VID_DIR, exist_ok=True)

In [36]:
# Verificar se os diretórios e locais de trabalho foram criados corretamente
# os.listdir(BASE_DIR)

In [7]:
# Definir contexto -período- de trabalho 

now = datetime.datetime.now()
year = datetime.datetime.now().year
day = datetime.datetime.now().day
month = datetime.datetime.now().month

# Usar contexto para nomear documentos

context = f"_{month}-{day}-{year}"

In [38]:
#print(context) """retirar comentário para ver resultado"""

## Configurar Youtube API

In [8]:
# Configurar YouTube API

api_key = "" # inserir aqui a sua chave - "api key"

# Biblioteca 
from googleapiclient.discovery import build

# Conf. básica
youtube = build("youtube", "v3", developerKey=api_key)

### Pegar os dados básicos de Canal de YouTube

- levantar links de todos os vídeos do canal
- levantar dados e metadados de todos os vídeos do canal 

In [9]:
# Função selecionar - com base em id - todos os videos de um Canal do Youtube

def get_channel_videos(channel_id):
    
    # get Uploads playlist id
    
    res = youtube.channels().list(id=channel_id, 
                                  part='contentDetails').execute()
    playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    
    videos = []
    next_page_token = None
    
    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id, 
                                           part='snippet', 
                                           maxResults=50,
                                           pageToken=next_page_token).execute()
       
        videos += res['items']
        next_page_token = res.get('nextPageToken')
        
        if next_page_token is None:
            break
    
    return videos

In [10]:
# Informações do Canal de Jair Bolsonaro - id do Canal 

id_jair = "UC8hGUtfEgvvnp6IaHSAg1OQ" # Como encontrar id do canal? A forma mais fácil: 1. Selecionar qualquer vídeo do canal desejado.2.Clicar no logo/nome do canal abaixo do vídeo. 3. Selecionar a última parte do link do seu navegador. Exemplo: youtube.com/channel/UC8hGUtfEgvvnp6IaHSAg1OQ)
ContentDetails = youtube.channels().list(id = id_jair, part = "contentDetails").execute() # Para saber mais: https://developers.google.com/youtube/v3/docs/search/list 


In [11]:
# "Pegar" informações básicas de todos os vídeos de um Canal do Youtube. No caso, de Jair Bolsonaro
vids = get_channel_videos(id_jair)

In [43]:
# Verificar no primeiro ítem[0] quais são os dados levantados

#vids[0] """retirar comentário para ver resultado"""

In [12]:
# Levantar dados e metadados de todos os vídeos do canal


dados = []

for i in vids:
    ChannelTitle = i["snippet"]['channelTitle']
    PublishedAt = i["snippet"]['publishedAt']
    Title = i["snippet"]['title']
    Description = i["snippet"]['description'] 
    Id = i['snippet']['resourceId']['videoId']
    Link = "https://www.youtube.com/watch?v=" +i['snippet']['resourceId']['videoId']
    dados.append((ChannelTitle, PublishedAt, Title, Description, Id, Link))

dados = pd.DataFrame(dados, columns = ['ChannelTitle', 'PublishedAt','Title', 'Description', 'Id', 'Link'])

In [13]:
# Salvar em um documento CSV - JairChannel_corpus

title = f"{DATA_DIR}/JairChannel_corpus{context}.csv"
dados.to_csv(f"{title}", index=False) #dados.to_csv(f"{DATA_DIR}/JairChannel_vids_generaldata_{context}.csv", index=False)

### Verificar o corpus levantado - videos de um Canal do Youtube

- Abrir documento JairChannel_corpus
- Explorar dados gerais

    Para saber mais: https://paulovasconcellos.com.br/28-comandos-%C3%BAteis-de-pandas-que-talvez-voc%C3%AA-n%C3%A3o-conhe%C3%A7a-6ab64beefa93

In [14]:
# Abrir documento - Pandas Data Frame (df)
df = pd.read_csv(f"{title}")

In [15]:
# Qual o tamanho do documento - n. de itens?
len(df)

2639

In [16]:
# Quantas linhas e colunas o DataFrame tem?
df.shape

(2639, 6)

In [17]:
# Quais são as categorias levantadas/obsevadas?
df.columns

Index(['ChannelTitle', 'PublishedAt', 'Title', 'Description', 'Id', 'Link'], dtype='object')

In [18]:
# Qual a quantidade de intens em cada uma das categorias obsevadas - colunas?
df.count()

ChannelTitle    2639
PublishedAt     2639
Title           2639
Description      878
Id              2639
Link            2639
dtype: int64

In [19]:
# Quais são os 5 'primeiros' itens do documento - Os vídeos publicados recentemente?
df.head(5)

,ChannelTitle,PublishedAt,Title,Description,Id,Link
0,Jair Bolsonaro,2021-03-03T12:41:36Z,Piauí: o Governo do Brasil entregou nesta terç...,No subtrecho que vai do povoado de Estaca Zero...,t4biNqt_v1o,https://www.youtube.com/watch?v=t4biNqt_v1o
1,Jair Bolsonaro,2021-03-02T18:45:53Z,Liberadas as águas do Eixo Norte da Integração...,O Ministério do Desenvolvimento Regional liber...,GMzbTvZhnAI,https://www.youtube.com/watch?v=GMzbTvZhnAI
2,Jair Bolsonaro,2021-03-02T02:04:57Z,"- Combustíveis, tabela de imposto de renda, tu...",NaN,9jhTU5xLWU0,https://www.youtube.com/watch?v=9jhTU5xLWU0
3,Jair Bolsonaro,2021-03-01T08:33:52Z,"- Diretamente da Índia, o lançamento o Amazôni...",- Sevirá também para monitorar nossa Amazônia....,_gQhQYP0w-k,https://www.youtube.com/watch?v=_gQhQYP0w-k
4,Jair Bolsonaro,2021-02-28T22:34:07Z,- O povo quer trabalhar. - Brasília/DF.,NaN,IdfoqQzcpbo,https://www.youtube.com/watch?v=IdfoqQzcpbo


In [20]:
# Quais são os 5 'últimos itens' do documento - Os primeiros vídeos publicados no canal?
df.tail(5)

,ChannelTitle,PublishedAt,Title,Description,Id,Link
2634,Jair Bolsonaro,2010-11-26T14:52:56Z,FAMILIA NA VISÃO DOS GAYS,NaN,ZizcEN_4x9w,https://www.youtube.com/watch?v=ZizcEN_4x9w
2635,Jair Bolsonaro,2010-11-25T16:30:39Z,Bolsonaro defende a família.,NaN,uG-MvfE3nCs,https://www.youtube.com/watch?v=uG-MvfE3nCs
2636,Jair Bolsonaro,2010-11-18T13:21:49Z,Segurança Pública,Por que eles matam?\r\nPorque nós somos covardes.,xZWwiJjGsao,https://www.youtube.com/watch?v=xZWwiJjGsao
2637,Jair Bolsonaro,2010-11-12T12:08:14Z,BOLSONARO E PNDH3,NaN,EWzeqhITpoM,https://www.youtube.com/watch?v=EWzeqhITpoM
2638,Jair Bolsonaro,2010-11-11T12:40:16Z,BOLSONARO FALA EM VEADOS E CITA DATENA,NaN,UkSBwk1Efp8,https://www.youtube.com/watch?v=UkSBwk1Efp8
